# Generate SC scores for qm9 molecules

In [6]:
import numpy as np
import pandas as pd
from p_tqdm import p_map
from sa_sc_syba_score_parallel import *
%matplotlib inline

## Read cleaned qm9 data

In [7]:
qm9_filepath = '../Computing Emin for QM9/data/clean/qm9.csv'
mol_data = pd.read_csv(qm9_filepath)
os.makedirs('sc_score_sublists',exist_ok=True)

In [8]:
def parallelize_dataframe(df,func,n_splits=100,n_cores=4):
    # taken from https://towardsdatascience.com/make-your-own-super-pandas-using-multiproc-1c04f41944a1
    model = SCScorePrecursorPrioritizer()
    model.load_model(model_tag='1024bool')
    df_split = list(zip(np.array_split(df,n_splits),range(n_splits),[model]*n_splits))
    p_map(func,df_split,num_cpus = n_cores)

## Calculate SC scores into mol_data dataframe and overwrite cleaned qm9 data

In [ ]:
parallelize_dataframe(mol_data['smiles_1'],parallel_apply_sc_score,n_cores=4)
mol_data['smiles_1 sc_score'] = np.concatenate([np.loadtxt(f'sc_score_sublists/sc_score_{i}.txt') for i in range(100)])
mol_data.to_csv(qm9_filepath)